2020-01-24: something was wrong with the skw routine in tasks_mpi. Turned out that we are 
still calculating the conditional spectra one-by-one for each channel.
Some indentation was missing

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import sys
sys.path.append("/global/homes/r/rkube/repos/delta")
import numpy as np
import json

In [9]:
#from analysis.tasks_mpi import bicoherence
from analysis.task_fft import task_fft_scipy

In [10]:
from analysis.channels import channel, channel_range, channel_pair

In [11]:
with open("../configs/test_skw.json") as df:
    cfg = json.load(df)

In [26]:
# Create an fft object and generate fft data as in processor_mpi
cfg["fft_params"]["fsample"] = cfg["ECEI_cfg"]["SampleRate"] * 1e3
my_fft = task_fft_scipy(10_000, cfg["fft_params"], normalize=True, detrend=True)

fft_params = my_fft.get_fft_params()

In [13]:
data = np.random.uniform(0.0, 1.0, [192, 10_000])

In [14]:
fft_data = my_fft.do_fft_local(data)

In [15]:
fft_data.shape

(192, 512, 38)

In [16]:
# Create a dummy channel pair list

In [17]:
chrg_ref = channel_range.from_str(cfg["task_list"][0]["ref_channels"])
chrg_cmp = channel_range.from_str(cfg["task_list"][0]["cmp_channels"])

chpair_list = [channel_pair(ch1, ch2) for ch1, ch2 in zip(chrg_ref, chrg_cmp)]

In [32]:
def skw(fft_data, ch_it, fft_params, ecei_config, info_dict, kstep=0.01): 
    """
    Calculates the conditional spectrum S(k,w).

    Input:
    ======
    fft_data: dask_array, float: Contains the fourier-transformed data. dim0: channel, dim1: Fourier Coefficients
    ch0: channel, first channel
    ch1: channel, second channel
    fft_params: dictionary, parameters for fft
    ecei_config: dictionary, configuration of ecei diagnostic


    Returns:
    ========
    bicoherence, float.
    """

    from analysis.ecei_helper import channel_position

    res_list = []
    for ch_pair in ch_it:
        ch1 = ch_pair.ch1
        ch2 = ch_pair.ch2
        ch1_idx, ch2_idx = ch1.idx(), ch2.idx()
        #logging.debug("Calculating skw for channels {0:s}x{1:s}".format(ch_pair.ch1, ch_pair.ch2))

        XX = np.fft.fftshift(fft_data[ch1_idx, :, :], axes=0).T
        YY = np.fft.fftshift(fft_data[ch2_idx, :, :], axes=0).T

        bins, _ = XX.shape
        win_factor = fft_params["win_factor"]

        cpos_ref = channel_position(ch1, ecei_config)
        cpos_cmp = channel_position(ch2, ecei_config)

        # Calculate distance between channels
        dist = np.sqrt( (cpos_ref[0] - cpos_cmp[0])**2.0 + (cpos_ref[1] - cpos_cmp[1])**2.0)
        dmin = dist * 1e2

        kax = np.arange(-np.pi / dmin, np.pi / dmin, kstep)

        nkax = kax.size
        nfft = fft_params["nfft"]

        if(ch1_idx == ch2_idx):
            # We can't calculate the cross-conditional spectrum for ch0==ch1
            # since dmin=0
            return(np.zeros(nkax, nfft))

        # value dimension
        Pxx = np.zeros((bins, nfft), dtype=np.complex_)
        Pyy = np.zeros((bins, nfft), dtype=np.complex_)
        Kxy = np.zeros((bins, nfft), dtype=np.complex_)
        val = np.zeros((nkax, nfft), dtype=np.complex_)

        sklw = np.zeros((nkax, nfft), dtype=np.complex_)
        K = np.zeros(nfft, dtype=np.complex_)
        sigK = np.zeros(nfft, dtype=np.complex_)


        #logging.debug(f"nkax = {nkax:d}, nfft = {nfft:d}, dmin = {dmin:f}")

        # calculate auto power and cross phase (wavenumber)
        for b in range(bins):
            #X = self.Dlist[done].spdata[done_subset[c],b,:]
            #Y = self.Dlist[dtwo].spdata[dtwo_subset[c],b,:]
            X = XX[b, :]
            Y = YY[b, :]

            Pxx[b,:] = X*np.matrix.conjugate(X) / win_factor
            Pyy[b,:] = Y*np.matrix.conjugate(Y) / win_factor
            Pxy = X*np.matrix.conjugate(Y)
            Kxy[b,:] = np.arctan2(Pxy.imag, Pxy.real).real / (dist * 100) # [cm^-1]

            # calculate SKw
            for w in range(nfft):
                idx = (Kxy[b,w] - kstep * 0.5 < kax) * (kax < Kxy[b,w] + kstep * 0.5)
                val[:,w] = val[:,w] + (1.0 / bins * (Pxx[b,w] + Pyy[b,w]) * 0.5) * idx

        # calculate moments
        sklw = val / np.tile(val.sum(axis=0), (nkax, 1))
        K[:] = np.sum(np.transpose(np.tile(kax, (nfft, 1))) * sklw, axis=0)
        for w in range(nfft):
            sigK[w] = np.sqrt(np.sum( (kax - K[w])**2 * sklw[:,w] ))

        val = val.mean(axis=0).real
        K = np.mean(K, axis=0)
        sigK = np.mean(sigK, axis=0)
        pdata = np.log10(val + 1e-10)

        res_list.append(pdata)

    return(res_list, info_dict)

In [33]:
skw(fft_data, chpair_list, fft_params, cfg['ECEI_cfg'], None)

([array([-4.86187899, -4.92238046, -4.93350713, -4.91462088, -4.94550165,
         -4.88016727, -4.87597806, -4.93831684, -4.90870313, -4.96831875,
         -4.99731923, -4.93523958, -4.94016383, -4.96765142, -4.96724773,
         -4.98115613, -4.91630103, -4.91013538, -4.93305994, -4.97879975,
         -5.03743858, -4.93829985, -4.95133745, -4.94390398, -4.9788669 ,
         -4.97163954, -4.90809547, -4.89760584, -4.910321  , -4.90916744,
         -4.8842267 , -4.93454242, -5.03418509, -5.00978406, -4.91270233,
         -4.92516613, -4.90203359, -4.8851228 , -4.93621444, -5.05465463,
         -5.07039093, -4.96112417, -4.99762711, -5.05048257, -5.02633639,
         -4.96505988, -4.99195904, -4.96643607, -4.97119468, -5.02697231,
         -4.93969452, -4.90426149, -4.89295044, -4.89747693, -4.87450596,
         -4.86404754, -4.95229292, -4.99517464, -4.91691648, -4.95151056,
         -4.98521922, -5.0314929 , -4.99000565, -4.82928233, -4.94184051,
         -4.92713065, -4.99654796, -4.

In [42]:
def coherence(fft_data, ch_it, fft_config, info_dict):
    """Kernel that calculates the coherence between two channels.
    Input:
    ======    
    fft_data: ndarray, float: Contains the fourier-transformed data. 
                dim0: channel, dim1: Fourier Coefficients. dim2: STFT (bins in fluctana code)
    ch_it: iterable, Iterator over a list of channels we wish to perform our computation on


    Returns:
    ========
    coherence, float.
    """

    c1_idx = np.array([ch_pair.ch1.idx() for ch_pair in ch_it])
    c2_idx = np.array([ch_pair.ch2.idx() for ch_pair in ch_it])

    X = fft_data[c1_idx, :, :]
    Y = fft_data[c2_idx, :, :]

    Pxx = X * X.conj()
    Pyy = Y * Y.conj()

    Gxy = np.mean((X * Y.conj()) / np.sqrt(Pxx * Pyy), axis=2)
    
    #print(Gxy)
    #Gxy = np.fabs(Gxy.real)

    return(Gxy, info_dict)

In [48]:
GG, _ = coherence(fft_data, chpair_list, fft_params, cfg['ECEI_cfg']);

In [56]:
np.fabs(GG.real)

array([[0.05263158, 0.1705434 , 0.03399223, 0.09730163, 0.1151428 ,
        0.04514039, 0.02582669, 0.04684854, 0.111333  , 0.00809159,
        0.0078383 , 0.1420049 , 0.14719808, 0.16388938, 0.12367904,
        0.1120856 , 0.03497395, 0.17878891, 0.04959764, 0.2273268 ,
        0.13756835, 0.00813338, 0.21021674, 0.23607948, 0.0524827 ,
        0.07427034, 0.02956471, 0.12235664, 0.00750519, 0.04539071,
        0.17916188, 0.15265389, 0.19430126, 0.23531223, 0.15693133,
        0.18826967, 0.0451555 , 0.05531797, 0.10789615, 0.05386624,
        0.03803497, 0.16346597, 0.03027807, 0.0820435 , 0.09863577,
        0.19532314, 0.14523585, 0.12383789, 0.38949928, 0.03035503,
        0.15306313, 0.12374617, 0.20714476, 0.11333828, 0.29520554,
        0.19653101, 0.14381102, 0.18917168, 0.15711477, 0.05406886,
        0.06412951, 0.0140791 , 0.06638046, 0.09204417, 0.15601659,
        0.09066992, 0.17675651, 0.0538422 , 0.02239427, 0.11734348,
        0.01167855, 0.04430294, 0.02274227, 0.06

In [52]:
GG.dtype

dtype('complex128')

In [54]:
?np.abs

Call signature:  np.abs(*args, **kwargs)
Type:            ufunc
String form:     <ufunc 'absolute'>
File:            ~/.conda/envs/delta/lib/python3.7/site-packages/numpy/__init__.py
Docstring:      
absolute(x, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])

Calculate the absolute value element-wise.

``np.abs`` is a shorthand for this function.

Parameters
----------
x : array_like
    Input array.
out : ndarray, None, or tuple of ndarray and None, optional
    A location into which the result is stored. If provided, it must have
    a shape that the inputs broadcast to. If not provided or `None`,
    a freshly-allocated array is returned. A tuple (possible only as a
    keyword argument) must have length equal to the number of outputs.
where : array_like, optional
    This condition is broadcast over the input. At locations where the
    condition is True, the `out` array will be set to the ufunc result.
    Elsewhere, the `o